## 在SD21上测试PAN攻击

In [1]:
class Args():
    def __init__(self):
        self.dataset_name = 'VGGFace2-clean'
        self.instance_name = '0'
        self.advance_steps = 2
        self.target_weight = 1
        self.device = 2
        self.exp_name = 'release-MetaCloak-advance_steps-2-total_trail_num-1-unroll_steps-1-interval-2-total_train_steps-10-SD21base-robust-gauK-7'
        self.exp_hyper = 'dataset-VGGFace2-clean-r-11-model-SD21base-gen_prompt-sks'
        self.pretrained_model_name_or_path = '/data/home/yekai/github/mypro/MetaCloak/SD/stable-diffusion-2-1-base'
        self.revision = None
        self.tokenizer_name = None
        self.class_data_dir = '/data/home/yekai/github/mypro/MetaCloak/prior-data/SD21base/class-person'
        self.instance_prompt = 'a photo of sks person'
        self.class_prompt = 'a photo of person'
        self.with_prior_preservation = True
        self.prior_loss_weight = 1.0
        self.num_class_images = 200
        self.output_dir = '/data/home/yekai/github/mypro/MetaCloak/exp_data/gen_output/release-MetaCloak-advance_steps-2-total_trail_num-1-unroll_steps-1-interval-2-total_train_steps-10-SD21base-robust-gauK-7/dataset-VGGFace2-clean-r-11-model-SD21base-gen_prompt-sks/0'
        self.seed = 0
        self.resolution = 512
        self.center_crop = True
        self.train_text_encoder = True
        self.train_batch_size = 1
        self.sample_batch_size = 8
        self.max_train_steps = 20
        self.checkpointing_iterations = 10
        self.learning_rate = 5e-07
        self.logging_dir = 'logs'
        self.allow_tf32 = False
        self.report_to = 'wandb'
        self.mixed_precision = 'fp16'
        self.enable_xformers_memory_efficient_attention = True
        self.wandb_entity_name = 'vkeilo'
        self.wandb_run_name = 'MAT-PAN-10-2-1-1-x1x1-radius11-noSGLD-robust0-10-0.01-0.5-k=2-useS-last-1728469324'
        self.wandb_project_name = 'metacloak_PAN'
        self.transform_hflip = True
        self.instance_data_dir_for_train = '/data/home/yekai/github/mypro/MetaCloak/dataset/VGGFace2-clean/0/set_A'
        self.instance_data_dir_for_adversarial = '/data/home/yekai/github/mypro/MetaCloak/dataset/VGGFace2-clean/0/set_B'
        self.defense_pgd_ascending = True
        self.defense_pgd_radius = 11.0
        self.defense_pgd_step_size = 1.1
        self.defense_pgd_step_num = 1
        self.defense_pgd_random_start = False
        self.attack_pgd_radius = 11.0
        self.attack_pgd_step_size = 0.3
        self.attack_pgd_step_num = 6
        self.attack_pgd_ascending = False
        self.attack_pgd_random_start = False
        self.target_image_path = None
        self.gau_kernel_size = 7
        self.defense_sample_num = 1
        self.rot_degree = 5
        self.transform_rot = False
        self.transform_gau = True
        self.original_flow = False
        self.total_trail_num = 1
        self.unroll_steps = 1
        self.interval = 2
        self.total_train_steps = 10
        self.beta_s = 0.3
        self.beta_p = 0.3
        self.sampling_times_theta = 1
        self.sampling_times_delta = 1
        self.sampling_step_theta = 1e-05
        self.sampling_step_delta = 1e-05
        self.sampling_noise_ratio = 0.05
        self.attack_mode = 'pan'
        self.pan_lambda_D = 0.01
        self.pan_lambda_S = 10.0
        self.pan_omiga = 0.5
        self.pan_k = 2
        self.pan_mode = 'S'
        self.init_model_state_pool_pth_path = '/data/home/yekai/github/mypro/MetaCloak/robust_facecloak/attacks/algs/tmpdata/init_model_state_pool_sd2-1.pth'
        self.pan_use_val = 'last'
        self.model_select_mode = 'order'
        self.total_gan_step = 0
        self.SGLD_method = 'noSGLD'

args = Args()

In [2]:
import sys
sys.path.append('/data/home/yekai/github/mypro')
sys.path.append('/data/home/yekai/github/mypro/MetaCloak')

import random
import wandb
import argparse
import copy
import hashlib
import itertools
import logging
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from pathlib import Path
import datasets
import diffusers
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from diffusers import AutoencoderKL, DDPMScheduler, DiffusionPipeline, UNet2DConditionModel
from diffusers.utils.import_utils import is_xformers_available
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm import tqdm
from transformers import AutoTokenizer, PretrainedConfig
from robust_facecloak.model.db_train import  DreamBoothDatasetFromTensor
from robust_facecloak.model.db_train import import_model_class_from_model_name_or_path
from robust_facecloak.generic.data_utils import PromptDataset, load_data
from robust_facecloak.generic.share_args import share_parse_args
# vkeilo add it
import utils
import GPUtil
import time
import pickle
logger = get_logger(__name__)

import torch
import torch.nn.functional as F

# vkeilo add
device_g = torch.device("cuda")
# 针对模型的unet和文本编码器进行的训练
def train_few_step(
    args,
    models,
    tokenizer,
    noise_scheduler,
    vae,
    data_tensor: torch.Tensor,
    num_steps=20,
    step_wise_save=False,
    save_step=100, 
    retain_graph=False,
    dpcopy = True,
    task_loss_name = None,
    loss_return = False,
):
    # Load the tokenizer
    # vkeilo remove deepcopy
    if dpcopy:
        unet, text_encoder = copy.deepcopy(models[0]), copy.deepcopy(models[1])
    else:
        unet, text_encoder = models[0], models[1]
    # unet, text_encoder = models[0], models[1]
    # 绑定unet和文本编码器的参数，共同优化
    params_to_optimize = itertools.chain(unet.parameters(), text_encoder.parameters())

    # 设置优化器，优化目标为unet参数和文本编码器参数
    optimizer = torch.optim.AdamW(
        params_to_optimize,
        lr=args.learning_rate,
        betas=(0.9, 0.999),
        weight_decay=1e-2,
        eps=1e-08,
    )

    train_dataset = DreamBoothDatasetFromTensor(
        data_tensor,
        # A photo of sks person
        args.instance_prompt,
        tokenizer,
        args.class_data_dir,
        args.class_prompt,
        args.resolution,
        args.center_crop,
    )

    weight_dtype = torch.bfloat16
    device = torch.device("cuda")

    # 将关键模型移动到对应设备
    vae.to(device, dtype=weight_dtype)
    text_encoder.to(device, dtype=weight_dtype)
    unet.to(device, dtype=weight_dtype)

    
    step2modelstate={}
        
    pbar = tqdm(total=num_steps, desc="training")
    for step in range(num_steps):
        # 根据设置选择是否保存训练中间过程参数
        if step_wise_save and ((step+1) % save_step == 0 or step == 0):
            # make sure the model state dict is put to cpu
            step2modelstate[step] = {
                "unet": copy.deepcopy(unet.cpu().state_dict()),
                "text_encoder": copy.deepcopy(text_encoder.cpu().state_dict()),
            }
            # move the model back to gpu
            unet.to(device, dtype=weight_dtype); text_encoder.to(device, dtype=weight_dtype)
            
        pbar.update(1)
        # 训练模式
        unet.train()
        text_encoder.train()
        # 循环从训练数据集中取一个样本
        step_data = train_dataset[step % len(train_dataset)]
        # 将样本中的类别图片和实例图片整合并移动到设备上
        pixel_values = torch.stack([step_data["instance_images"], step_data["class_images"]]).to(
            device, dtype=weight_dtype
        )
        # 将样本中的类别提示词和实例提示词整合并移动到设备上
        input_ids = torch.cat([step_data["instance_prompt_ids"], step_data["class_prompt_ids"]], dim=0).to(device)
        # 使用VAE对图像进行编码，并对潜在表示进行后处理
        latents = vae.encode(pixel_values).latent_dist.sample()
        latents = latents * vae.config.scaling_factor
        # print(f'latents shape: {latents.shape}')
        # Sample noise that we'll add to the latents
        # 向图片编码向量（潜在空间向量表示）添加随机噪声
        noise = torch.randn_like(latents)
        # batch_size
        bsz = latents.shape[0]
        # Sample a random timestep for each image
        # 为每个图片生成一个随机step
        timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bsz,), device=latents.device)
        timesteps = timesteps.long()

        # Add noise to the latents according to the noise magnitude at each timestep
        # (this is the forward diffusion process)
        # 前向过程，得到前向扩散特定时间步后的图片的潜在空间向量
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Get the text embedding for conditioning
        # 文本编码向量作为条件信息
        encoder_hidden_states = text_encoder(input_ids)[0]
        
        # Predict the noise residual
        # 模型基于当前的噪声潜在表示（noisy_latents）、时间步（timesteps）和文本条件（encoder_hidden_states），预测噪声残差
        # print(f'noisy_latents shape: {noisy_latents.shape}')
        model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
        # print('model_pred shape', model_pred.shape)
        # Get the target for loss depending on the prediction type
        # 预测的可以是噪声，也可以是变化速度
        if noise_scheduler.config.prediction_type == "epsilon":
            target = noise
        elif noise_scheduler.config.prediction_type == "v_prediction":
            target = noise_scheduler.get_velocity(latents, noise, timesteps)
        else:
            raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

        if args.with_prior_preservation:
            # 再次分为一半一半，对应之前的stack操作
            model_pred, model_pred_prior = torch.chunk(model_pred, 2, dim=0)
            target, target_prior = torch.chunk(target, 2, dim=0)

            # Compute instance loss
            instance_loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

            # Compute prior loss  确保在原来类别上的生成能力不丢失
            prior_loss = F.mse_loss(model_pred_prior.float(), target_prior.float(), reduction="mean")

            # Add the prior loss to the instance loss.
            loss = instance_loss + args.prior_loss_weight * prior_loss

        else:
            # 不使用先验保留损失
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")
        # 反向传播
        loss.backward(retain_graph=retain_graph)
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(params_to_optimize, 1.0, error_if_nonfinite=True)
        # 参数优化
        optimizer.step()
        optimizer.zero_grad()
    pbar.close()
    # 返回训练的参数数据
    if not loss_return:
        if step_wise_save:
            return [unet, text_encoder], step2modelstate
        else:     
            return [unet, text_encoder]
    else:
        if step_wise_save:
            return [unet, text_encoder], step2modelstate, loss.item()
        else:     
            return [unet, text_encoder], loss.item()

# 主要模型的加载
def load_model(args, model_path):
    print(f'out {model_path}')
    # import correct text encoder class
    text_encoder_cls = import_model_class_from_model_name_or_path(model_path, args.revision)

    # Load scheduler and models
    # 文本编码器加载
    text_encoder = text_encoder_cls.from_pretrained(
        model_path,
        subfolder="text_encoder",
        revision=args.revision,
    )
    # unet加载
    unet = UNet2DConditionModel.from_pretrained(model_path, subfolder="unet", revision=args.revision)

    # vkeilo add it
    # text_encoder = text_encoder.bfloat16()
    # unet = unet.bfloat16()

    # tokenizer加载
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        subfolder="tokenizer",
        revision=args.revision,
        use_fast=False,
    )
    # 使用DDPM同款调度器
    noise_scheduler = DDPMScheduler.from_pretrained(model_path, subfolder="scheduler")
    # 加载预训练的vae，vae不需要更新参数
    vae = AutoencoderKL.from_pretrained(model_path, subfolder="vae", revision=args.revision)

    vae.requires_grad_(False)

    # 甚至可以不更新文本编码器的参数
    if not args.train_text_encoder:
        text_encoder.requires_grad_(False)

    if args.enable_xformers_memory_efficient_attention:
        print("You selected to used efficient xformers")
        print("Make sure to install the following packages before continue")
        print("pip install triton==2.0.0.dev20221031")
        print("pip install pip install xformers==0.0.17.dev461")

        unet.enable_xformers_memory_efficient_attention()
    # 返回5个关键模型
    return text_encoder, unet, tokenizer, noise_scheduler, vae

/data/home/yekai/apps/miniconda/envs/Metacloak/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-24 10:26:17.679637: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 10:26:17.679711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 10:26:17.681329: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 10:26:17.688819: I tensorflow/core/platform/cpu_feature_guard.cc:182] This

In [3]:
def find_max_pixel_change(original_img, noisy_img):
    diff = torch.abs(original_img - noisy_img)

    # Find the maximum pixel difference
    max_change = torch.max(diff)

    return max_change.item()

In [4]:
raise ValueError("wait here")

ValueError: wait here

In [5]:

# 确保SGLD方法参数合法
assert args.SGLD_method in ["allSGLD", "thetaSGLD", "deltaSGLD" ,"noSGLD"]
# 指定日志目录
logging_dir = Path(args.output_dir, args.logging_dir)
# Hugging Face加速器，指定混合精度训练模式和记录方式，默认为wandb
accelerator = Accelerator(
    mixed_precision=args.mixed_precision,
    log_with=args.report_to,
    # logging_dir=logging_dir,
)

# 初始化日志记录器，指定格式和日志级别
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
# 记录加速器信息
logger.info(accelerator.state, main_process_only=False)
# 只在主进程上尽可能详细地记录日志
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_warning()
    diffusers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()
    diffusers.utils.logging.set_verbosity_error()
# 设置随机种子
if args.seed is not None:
    set_seed(args.seed)

# Generate class images if prior preservation is enabled.
# 如果启用了先验保留，则先生成类图像，保存在变量cur_class_images中
if args.with_prior_preservation:
    # 检查并创建一个用于存储类别图像的目录，并统计当前目录中已经存在的类别图像数量
    class_images_dir = Path(args.class_data_dir)
    if not class_images_dir.exists():
        class_images_dir.mkdir(parents=True)
    cur_class_images = len(list(class_images_dir.iterdir()))
    # 如果当前类别图像数量小于所需数量，则生成新的类别图像
    if cur_class_images < args.num_class_images:
        torch_dtype = torch.float16 if accelerator.device.type == "cuda" else torch.float32
        if args.mixed_precision == "fp32":
            torch_dtype = torch.float32
        elif args.mixed_precision == "fp16":
            torch_dtype = torch.float16
        elif args.mixed_precision == "bf16":
            torch_dtype = torch.bfloat16
        # 此处的pipline是用来根据类别提示生成类别图像的模型
        pipeline = DiffusionPipeline.from_pretrained(
            list(args.pretrained_model_name_or_path.split(","))[-1], 
            torch_dtype=torch_dtype,
            safety_checker=None,
            revision=args.revision,
        )
        # 生成类别图像时，不显示进度条
        pipeline.set_progress_bar_config(disable=True)
        # 计算还需要生成的类别图像数量，num_class_images默认为200
        num_new_images = args.num_class_images - cur_class_images
        logger.info(f"Number of class images to sample: {num_new_images}.")
        # 类别生成的提示词数据（其实都是 a photo of a person，因为在隐私保护中，扩散模型的的先验能力就是在人像类别上的生成能力）
        sample_dataset = PromptDataset(args.class_prompt, num_new_images)
        # sample_batch_size默认为4
        sample_dataloader = torch.utils.data.DataLoader(sample_dataset, batch_size=args.sample_batch_size)
        # 将数据集使用accelerator进行预处理（就是设置为使用混合精度和wandb记录）
        sample_dataloader = accelerator.prepare(sample_dataloader)
        # 将模型移动到accelerator.device上（accelerator） 会自动选择
        pipeline.to(accelerator.device)

        # 每批4个提示词，生成4个类别图像（一样的提示词）
        for example in tqdm(
            sample_dataloader,
            desc="Generating class images",
            disable=not accelerator.is_local_main_process,
        ):
            images = pipeline(example["prompt"]).images

            for i, image in enumerate(images):
                # 使用哈希值为图像文件名，并保存
                hash_image = hashlib.sha1(image.tobytes()).hexdigest()
                image_filename = class_images_dir / f"{example['index'][i] + cur_class_images}-{hash_image}.jpg"
                image.save(image_filename)
        # 删除模型，释放显存
        del pipeline
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# 判断是否启用tf32加速
if args.allow_tf32:
    torch.backends.cuda.matmul.allow_tf32 = True

# 加载干净数据
clean_data = load_data(
    args.instance_data_dir_for_train,
    # size=args.resolution,
    # center_crop=args.center_crop,
)

# 加载原始扰动数据
perturbed_data = load_data(
    args.instance_data_dir_for_adversarial,
    # size=args.resolution,
    # center_crop=args.center_crop,
)

# original_data当前为原始扰动数据
original_data= copy.deepcopy(perturbed_data)
    
import torchvision
# 定义训练和测试时的数据增强（图像处理）
train_aug = [
        # 双线性插值到512x512
        transforms.Resize(args.resolution, interpolation=transforms.InterpolationMode.BILINEAR),
        # 裁剪
        transforms.CenterCrop(args.resolution) if args.center_crop else transforms.RandomCrop(args.resolution),
]
# 图像随机旋转
rotater = torchvision.transforms.RandomRotation(degrees=(0, args.rot_degree))
# 高斯模糊
gau_filter = transforms.GaussianBlur(kernel_size=args.gau_kernel_size,)
# 制定防御对抗样本变换策略
defense_transform = [
]
if args.transform_hflip:
    defense_transform = defense_transform + [transforms.RandomHorizontalFlip(p=0.5)]
if args.transform_rot:
    defense_transform = defense_transform + [rotater]
if args.transform_gau:
    defense_transform = [gau_filter] + defense_transform

# 标准化均值和标准差
tensorize_and_normalize = [
    transforms.Normalize([0.5*255]*3,[0.5*255]*3),
]

# 将所有变换组合起来，整合为一个Compose对象
all_trans = train_aug + defense_transform + tensorize_and_normalize
all_trans = transforms.Compose(all_trans)


from robust_facecloak.attacks.worker.robust_pgd_worker_vk import RobustPGDAttacker
from MetaCloak.robust_facecloak.attacks.worker.pgd_worker import PGDAttacker
from MetaCloak.robust_facecloak.attacks.worker.pan_worker import PANAttacker
# 构建攻击者和防御者，攻击者使用PGD算法
# attacker = PGDAttacker(
#     radius=args.attack_pgd_radius, 
#     steps=args.attack_pgd_step_num, 
#     step_size=args.attack_pgd_step_size,
#     random_start=args.attack_pgd_random_start,
#     ascending=args.attack_pgd_ascending,
#     args=args, 
#     x_range=[-1, 1],
# )
# defender = RobustPGDAttacker(
#     radius=args.defense_pgd_radius,
#     steps=args.defense_pgd_step_num, # 6
#     step_size=args.defense_pgd_step_size,
#     random_start=args.defense_pgd_random_start,
#     ascending=args.defense_pgd_ascending,
#     args=args,
#     attacker=attacker, 
#     trans=all_trans,
#     sample_num=args.defense_sample_num,
#     x_range=[0, 255],
# )
assert args.attack_mode in ['pgd','pan']
if args.attack_mode == 'pgd':
    attacker = PGDAttacker(
        radius=args.attack_pgd_radius, 
        steps=args.attack_pgd_step_num, 
        step_size=args.attack_pgd_step_size,
        random_start=args.attack_pgd_random_start,
        ascending=args.attack_pgd_ascending,
        args=args, 
        x_range=[-1, 1],
    )
    defender = RobustPGDAttacker(
        radius=args.defense_pgd_radius,
        steps=args.defense_pgd_step_num, # 6
        step_size=args.defense_pgd_step_size,
        random_start=args.defense_pgd_random_start,
        ascending=args.defense_pgd_ascending,
        args=args,
        attacker=attacker, 
        trans=all_trans,
        sample_num=args.defense_sample_num,
        x_range=[0, 255],
        # vkeilo add it
        # step_sample_num=args.sampling_times_delta
    )
else:
    attacker = PANAttacker(
        radius=args.attack_pgd_radius,
        steps=args.attack_pgd_step_num,
        step_size=args.attack_pgd_step_size,
        # ascending=args.defense_pgd_ascending,
        args=args,
        # trans=all_trans,
        # sample_num=args.defense_sample_num,
        x_range=[0, 255],
        lambda_D = args.pan_lambda_D,
        lambda_S = args.pan_lambda_S,
        k = args.pan_k,
        mode = args.pan_mode,
        use_val = args.pan_use_val,
    )
    attacker.weight_dtype = torch.bfloat16

# 模型加载，本次实验只有一个
print(f'args model path:{args.pretrained_model_name_or_path}')
model_paths = list(args.pretrained_model_name_or_path.split(","))
num_models = len(model_paths)

MODEL_BANKS = [load_model(args, path) for path in model_paths]

# 提取模型的文本编码器和UNet的状态字典
MODEL_STATEDICTS = [
    {
        "text_encoder": MODEL_BANKS[i][0].state_dict(),
        "unet": MODEL_BANKS[i][1].state_dict(),
    }
    for i in range(num_models)
]
# 此函数将保存经过扰动处理的图像数据到noise-ckpt/{id_stamp}目录中，id_stamp在此次实验中为迭代次数
def save_image(perturbed_data, id_stamp):
    save_folder = f"{args.output_dir}/noise-ckpt/{id_stamp}"
    os.makedirs(save_folder, exist_ok=True)
    noised_imgs = perturbed_data.detach()
    img_names = [
        str(instance_path).split("/")[-1]
        for instance_path in list(Path(args.instance_data_dir_for_adversarial).iterdir())
    ]
    for img_pixel, img_name in zip(noised_imgs, img_names):
        save_path = os.path.join(save_folder, f"noisy_{img_name}")
        Image.fromarray(
            img_pixel.float().detach().cpu().permute(1, 2, 0).numpy().squeeze().astype(np.uint8)
        ).save(save_path)


init_model_state_pool = {}
pbar = tqdm(total=num_models, desc="initializing models")
# split sub-models
# 对于每一个模型，都进行一次训练
for j in range(num_models):
    init_model_state_pool[j] = {}
    # 提取关键模块
    text_encoder, unet, tokenizer, noise_scheduler, vae = MODEL_BANKS[j]
    
    # 加载unet和text_encoder的模型参数
    unet.load_state_dict(MODEL_STATEDICTS[j]["unet"])
    text_encoder.load_state_dict(MODEL_STATEDICTS[j]["text_encoder"])
    # 打包unet和text_encoder
    f_ori = [unet, text_encoder]
    # 得到训练total_train_steps步之后的unet, text_encoder参数以及中间状态参数
    # print('start train few 702')
    if args.init_model_state_pool_pth_path is None:
        f_ori, step2state_dict = train_few_step(
                args,
                f_ori,
                tokenizer,
                noise_scheduler,
                vae,
                perturbed_data.float(),
                args.total_train_steps,
                step_wise_save=True,
                save_step=args.interval,
                task_loss_name="ori_model_train_loss",
        )  
        # init_model_state_pool就来保存训练中间状态参数
        init_model_state_pool[j] = step2state_dict
    else:
        pre_trained_pth_path = args.init_model_state_pool_pth_path.split(',')[j]
        print(f'model {j} use trained pth:{pre_trained_pth_path}')
        with open(pre_trained_pth_path, 'rb') as f:
            model_pth_dict = pickle.load(f)
            init_model_state_pool[j] = model_pth_dict[0]
    # 释放占用的资源
    del f_ori, unet, text_encoder, tokenizer, noise_scheduler, vae
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    pbar.update(1)
pbar.close()
# 提取保存的中间状态的step数据（1000，2000，3000.....）        
steps_list = list(init_model_state_pool[0].keys())
print(steps_list)
# 进度条，总train_few_step调用的次数*模型数量1*
pbar = tqdm(total=args.total_trail_num * num_models * (args.interval // args.advance_steps) * len(steps_list), desc="meta poison with model ensemble")

if args.total_gan_step == 0:
    total_gan_step = args.total_trail_num * num_models * (args.interval // args.advance_steps) * len(steps_list)
else:
    total_gan_step = args.total_gan_step
cnt=0
# vkeilo add it  确定噪声强度
theta_noise_epsion = args.sampling_step_theta * args.sampling_noise_ratio
delta_noise_epsion = args.sampling_step_delta * args.sampling_noise_ratio

def select_target_model(loss_log):
    # 选择loss最小的模型
    loss_mean_list = []
    for model_loss in loss_log:
        for steps,loss_list in model_loss.items():
            loss_mean_list.append(np.mean(loss_list))
    taget_index = np.argmin(loss_mean_list)
    model_i = taget_index // len(steps_list)
    split_step = steps_list[taget_index % len(steps_list)]
    return model_i, split_step
# learning perturbation over the ensemble of models
# 在多个模型集合上进行扰动优化
# 多次实验
# total_iterations = args.epochs * len(train_dataloader)
# model_state_num = steps_list*len(num_models)
assert args.model_select_mode in ['order','min_loss']
print(f"avalaible model num: {num_models},available steps: {str(steps_list)},total train step :{str(args.total_train_steps)}")
# raise ValueError("The number of classes must be greater than 1.")
# 如果是平均顺序选择模型池中的模型
if args.model_select_mode == 'order':
    for _ in range(args.total_trail_num):          
        # 针对每一个模型
        for model_i in range(num_models):
            print(f'using model {model_i}')
            # 确定关键组件
            start_time = time.time()
            text_encoder, unet, tokenizer, noise_scheduler, vae = MODEL_BANKS[model_i]
            # 对于每一个中间状态step
            for split_step in steps_list: 
                # 加载unet和文本编码器的中间状态参数
                unet.load_state_dict(init_model_state_pool[model_i][split_step]["unet"])
                text_encoder.load_state_dict(init_model_state_pool[model_i][split_step]["text_encoder"])
                f = [unet, text_encoder]
                # f = [unet.to(device_1), text_encoder.to(device_1)]
                
                # 每advance_steps步进行一次防御优化/对于每一组模型参数，进行200/2=100次对抗训练
                print(f'start {args.interval // args.advance_steps} times of defense optimization in step-{split_step} model')
                for j in range(args.interval // args.advance_steps):
                    # 更新一次扰动，使得扰动更加强大,后续需要在此处引入随机性（多轮采样优化），并以扰动的平均值作为后续的扰动
                    # vkeilo add it
                    mean_delta = perturbed_data.clone().detach()
                    print(f'start {args.sampling_times_delta} times of delta sampling ')
                    for k in range(args.sampling_times_delta):
                        print(f'sample delta {k}/{args.sampling_times_delta} times')
                        if args.attack_mode == "pgd":
                            perturbed_data,rubust_loss = defender.perturb(f, perturbed_data, original_data, vae, tokenizer, noise_scheduler,)
                        elif args.attack_mode == "pan":
                            perturbed_data,rubust_loss = attacker.attack(f, perturbed_data, original_data, vae, tokenizer, noise_scheduler,)
                        # wandb.log({"perturbedloss": rubust_loss})
                        # 此处引入随机梯度朗之万动力学
                        if args.SGLD_method == 'allSGLD' or args.SGLD_method == 'deltaSGLD':
                            perturbed_data = utils.SGLD(perturbed_data, args.sampling_step_delta, delta_noise_epsion).detach()
                        mean_delta = args.beta_s * mean_delta + (1 - args.beta_s) * perturbed_data
                    mean_delta.detach()
                    perturbed_data = mean_delta
                    print(f"max pixel change:{find_max_pixel_change(perturbed_data, original_data)}")
                    # f[0] = f[0].to(device_0)
                    # f[1] = f[1].to(device_0)
                    # perturbed_data = defender.perturb(f, perturbed_data, original_data, vae, tokenizer, noise_scheduler)
                    
                    # 扰动优化次数更新 +1
                    cnt+=1
                    # 在新的扰动数据下，训练advance_steps步，后续需要在此处引入随机性（多轮采样优化参数），并以参数的平均值作为模型的参数
                    back_parameters_list = [f[0].state_dict(),
                                            f[1].state_dict()]

                    mean_theta_list = [f[0].state_dict(),
                                    f[1].state_dict()]
                    
                    # print(f'start {args.sampling_times_theta} times of theta sampling')
                    for k in range(args.sampling_times_theta):
                        print(f'sample theta {k}/{args.sampling_times_theta} times')
                        f = train_few_step(
                            args,
                            f,
                            tokenizer,
                            noise_scheduler,
                            vae,
                            perturbed_data.float(),
                            args.advance_steps,
                            # device = device_1
                            dpcopy = False,
                            task_loss_name='model_theta_loss',
                        )
                        torch.cuda.empty_cache()
                        for model_index, model in enumerate(f):
                            # print(f"\nbefore culcu, GPU: {gpu.name}, Free Memory: {gpu.memoryFree / 1024:.2f} GB")
                            for name, p in model.named_parameters():
                                # 先尝试固定学习率的（因为迭代次数暂未确定）
                                # lr_now = lr_scheduler.get_last_lr()[0]
                                # 参数采样,引入随机性
                                if args.SGLD_method == 'allSGLD' or args.SGLD_method == 'thetaSGLD':
                                    p.data = utils.SGLD(p.data, args.sampling_step_theta, theta_noise_epsion)
                                # 模型参数也使用指数平均
                                # mean_theta_list[model_index][name] = args.beta_s * mean_theta_list[model_index][name] + (1 - args.beta_s) * p.data.to('cpu')
                                # mean_theta_list[model_index][name] = args.beta_s * mean_theta_list[model_index][name] + (1 - args.beta_s) * p.data
                                mean_theta_list[model_index][name].mul_(args.beta_s).add_((1 - args.beta_s) * p.data)
                            # print(f"\nafter calcu params, GPU: {gpu.name}, Free Memory: {gpu.memoryFree / 1024:.2f} GB")
                            # torch.cuda.empty_cache()
                    # lr_scheduler.step()
                    # 对于模型的unet和文本编码器，分别更新参数
                    for back_parameters, mean_theta in zip(back_parameters_list,mean_theta_list):
                        for name in back_parameters:
                            back_parameters[name] = args.beta_p * back_parameters[name] + (1 - args.beta_p) * mean_theta[name]
                            # back_parameters[name] = back_parameters[name].float()
                            # back_parameters[name].mul_(args.beta_p).add_((1 - args.beta_p) * mean_theta[name])
                    for index, model in enumerate(f):
                        # model.load_state_dict({k: v.to(device_g) for k, v in back_parameters_list[index].items()})
                        model.load_state_dict(back_parameters_list[index])
                        pass
                    del back_parameters_list
                    del mean_theta_list
                    gc.collect()
                    torch.cuda.empty_cache()
                    # f = train_few_step(
                    #     args,
                    #     f,
                    #     tokenizer,
                    #     noise_scheduler,
                    #     vae,
                    #     perturbed_data.float(),
                    #     args.advance_steps,
                    # )
                    pbar.update(1)
                    # 每1000次扰动优化，保存一次扰动示例图像
                    if cnt % 1000 == 0:
                        save_image(perturbed_data, f"{cnt}")
                # frequently release the memory due to limited GPU memory, 
                # env with more gpu might consider to remove the following lines for boosting speed
                # 释放资源
                del f 
                torch.cuda.empty_cache()
            end_time = time.time()
            # logger.info(f"model {model_i} adversarial training Time cost: {(end_time - start_time) / 60} min")
            # wandb.log({f"Time cost of model {model_i} adversarial training": (end_time - start_time) / 60})
            del unet, text_encoder, tokenizer, noise_scheduler, vae

            if torch.cuda.is_available():
                torch.cuda.empty_cache() 

        import gc
        gc.collect()
        torch.cuda.empty_cache()      

pbar.close()
# 保存最后的结果
# save_image(perturbed_data, "final")

print(f"max pixel change:{find_max_pixel_change(perturbed_data, original_data)}")



10/23/2024 17:08:34 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16



You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


args model path:/data/home/yekai/github/mypro/MetaCloak/SD/stable-diffusion-2-1-base
out /data/home/yekai/github/mypro/MetaCloak/SD/stable-diffusion-2-1-base
model_path:/data/home/yekai/github/mypro/MetaCloak/SD/stable-diffusion-2-1-base


{'conv_out_kernel', 'class_embed_type', 'mid_block_type', 'upcast_attention', 'timestep_post_act', 'projection_class_embeddings_input_dim', 'time_cond_proj_dim', 'time_embedding_type', 'resnet_time_scale_shift', 'conv_in_kernel'} was not found in config. Values will be initialized to default values.
{'clip_sample_range', 'variance_type'} was not found in config. Values will be initialized to default values.
{'scaling_factor'} was not found in config. Values will be initialized to default values.


You selected to used efficient xformers
Make sure to install the following packages before continue
pip install triton==2.0.0.dev20221031
pip install pip install xformers==0.0.17.dev461


...

start 1 times of defense optimization in step-799 model
start 1 times of delta sampling 
sample delta 0/1 times
epoch: 0, loss_S: 639.8359, loss_D: -0.0214
epoch: 1, loss_S: 719.9029, loss_D: -0.0267
epoch: 2, loss_S: 807.9678, loss_D: -0.0471
epoch: 3, loss_S: 899.9095, loss_D: -0.2615
epoch: 4, loss_S: 999.9913, loss_D: -0.0461
epoch: 5, loss_S: 1103.9666, loss_D: -0.0609
max pixel change:10.14251708984375
sample theta 0/1 times


meta poison with model ensemble:  83%|████████▎ | 5/6 [00:51<00:10, 10.22s/it]

start 1 times of defense optimization in step-999 model
start 1 times of delta sampling 
sample delta 0/1 times
epoch: 0, loss_S: 999.8782, loss_D: -0.0215
epoch: 1, loss_S: 1103.9403, loss_D: -0.0939
epoch: 2, loss_S: 1207.9139, loss_D: -0.0819
epoch: 3, loss_S: 1207.7570, loss_D: -0.0832
epoch: 4, loss_S: 1207.8892, loss_D: -0.1872
epoch: 5, loss_S: 1207.9585, loss_D: -0.2219
max pixel change:10.742759704589844
sample theta 0/1 times


meta poison with model ensemble: 100%|██████████| 6/6 [01:02<00:00, 10.40s/it]

max pixel change:10.742759704589844


In [5]:
init_model_state_pool

KeyboardInterrupt: 

In [8]:
from copy import deepcopy
# 提取保存的中间状态的step数据（0,199，399...）        
steps_list = list(init_model_state_pool[0].keys())
# 进度条，总train_few_step调用的次数
pbar = tqdm(total=args.total_trail_num * num_models * (args.interval // args.advance_steps) * len(steps_list), desc="meta poison with model ensemble")
cnt=0
# learning perturbation over the ensemble of models
# 在多个模型集合上进行扰动优化
# 多次实验
for _ in range(args.total_trail_num):
    # 针对每一个模型
    for model_i in range(num_models):
        # 确定关键组件
        text_encoder, unet, tokenizer, noise_scheduler, vae = MODEL_BANKS[model_i]
        # 对于每一个中间状态step
        for split_step in steps_list: 
            # 加载unet和文本编码器的中间状态参数
            unet.load_state_dict(init_model_state_pool[model_i][split_step]["unet"])
            text_encoder.load_state_dict(init_model_state_pool[model_i][split_step]["text_encoder"])
            f = [unet, text_encoder]
            # 每advance_steps步进行一次防御优化
            for j in range(args.interval // args.advance_steps):
                before = deepcopy(perturbed_data)
                perturbed_data,rubust_loss = attacker.attack(f, perturbed_data, original_data, vae, tokenizer, noise_scheduler)
                print(attacker.get_Linfty_norm(perturbed_data.to('cpu')-before.to('cpu')))
                print(attacker.get_Linfty_norm(perturbed_data.to('cpu')-original_data))
                # break
                # perturbed_data,rubust_loss = defender.perturb(f, perturbed_data, original_data, vae, tokenizer, noise_scheduler,)
                # 扰动优化次数更新 +1
                # wandb.log({"defender_rubust_loss_without_MAT": rubust_loss})
                cnt+=1
                
                f = train_few_step(
                    args,
                    f,
                    tokenizer,
                    noise_scheduler,
                    vae,
                    perturbed_data.float(),
                    args.advance_steps,
                    copy_flag = False,
                )
                pbar.update(1)
                # 每1000次扰动优化，保存一次扰动示例图像
                if cnt % 1000 == 0:
                    save_image(perturbed_data, f"{cnt}")
            
            # frequently release the memory due to limited GPU memory, 
            # env with more gpu might consider to remove the following lines for boosting speed
            # 释放资源
            del f 
            torch.cuda.empty_cache()
            # break
            
        del unet, text_encoder, tokenizer, noise_scheduler, vae

        if torch.cuda.is_available():
            torch.cuda.empty_cache() 
        # break
    import gc
    gc.collect()
    torch.cuda.empty_cache()   
    # break   
pbar.close()
# 保存最后的结果
save_image(perturbed_data, "final")

meta poison with model ensemble:   0%|          | 0/6 [00:09<?, ?it/s]


D:adv_image.grad.sign(): tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  .

TypeError: train_few_step() got an unexpected keyword argument 'copy_flag'